In [16]:
import torch
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
print(sys.path)

import Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.dynamics as dynamics
import Double_Pendulum.transforms as transforms

import Double_Pendulum.normal_form as normal_form

from datetime import datetime

rp = robot_parameters.LUMPED_PARAMETERS
rp["m1"] = 0.
model = autoencoders.Analytic_transformer(rp)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cw = False

['/home/kian/Documents/Thesis/ICS_fork/ics-pa-sv/Kian_code', '/home/kian/anaconda3/envs/thesis2/lib/python311.zip', '/home/kian/anaconda3/envs/thesis2/lib/python3.11', '/home/kian/anaconda3/envs/thesis2/lib/python3.11/lib-dynload', '', '/home/kian/anaconda3/envs/thesis2/lib/python3.11/site-packages', '/home/kian/anaconda3/envs/thesis2/lib/python3.11/site-packages/setuptools/_vendor', '/tmp/tmp_r29kzko']


In [17]:
xy_start = torch.tensor([4., -1.9]).requires_grad_().to(device)
q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)


xy_des_real = torch.tensor([2.1, -2.]).requires_grad_().to(device)
q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)

In [18]:
k_spring = 0.05

In [19]:
q_start = transforms.inverse_kinematics(xy_start, rp, is_clockwise=model_cw).unsqueeze(0)
is_clockwise_start = transforms.check_clockwise(q_start.squeeze(0))

q_des = transforms.inverse_kinematics(xy_des_real, rp, is_clockwise=model_cw).unsqueeze(0)
th_start = model.encoder_vmap(q_start)
th_d_start = (model.jacobian_enc(q_start) @ q_d_start.T).T
th_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Obtain accelerations in thetas. Do I care?

is_clockwise_des = transforms.check_clockwise(q_des.squeeze(0))

th_des = model.encoder_vmap(q_des)
th_d_des = (model.jacobian_enc(q_des) @ q_d_des.T).T


q_des_hat = model.decoder_vmap(th_des, is_clockwise_des)
q_d_des_hat = (model.jacobian_dec(th_des, clockwise=is_clockwise_des) @ th_d_des.T).T
xy_des_est, _ = transforms.forward_kinematics(rp, q_des_hat[0])

print(q_start)

tensor([[-0.8498,  0.2159]], device='cuda:0', grad_fn=<UnsqueezeBackward0>)


In [20]:
J_h_inv_des = model.jacobian_dec(th_des, is_clockwise_des).squeeze(0)
J_h_inv_trans_des = torch.transpose(J_h_inv_des, 0, 1)

M_q_des, C_q_des, G_q_des = dynamics.dynamical_matrices(rp, q_des_hat.squeeze(0), q_d_des_hat.squeeze(0))
G_q_des = dynamics.add_spring_force_G_q(rp, q_des_hat, G_q_des, k_spring)
M_th_des, C_th_des, G_th_des = transforms.transform_dynamical_from_inverse(M_q_des, C_q_des, G_q_des, th_des, th_d_des, J_h_inv_des, J_h_inv_trans_des)

K = torch.tensor([[10., 100., 100., 100.]]).to(device)
Y_des_u = normal_form.calculate_Y(th_des, th_d_des, M_th_des, C_th_des, G_th_des, device)
Y_des = torch.tensor([[Y_des_u[0,0]], [0], [0], [0]]).to(device)
print("Y_des_u:\n", Y_des_u)
print("Y_des:\n", Y_des)

Y_des_u:
 tensor([[ 1.5851],
        [ 0.0000],
        [-0.0218],
        [ 0.0000]], device='cuda:0')
Y_des:
 tensor([[1.5851],
        [0.0000],
        [0.0000],
        [0.0000]], device='cuda:0')


In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

th_series, th_d_series, th_dd_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_est_series, q_d_est_series, q_dd_est_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_real_series, q_d_real_series, q_dd_real_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
Y_series = torch.empty((0,4)).to(device)
u_series = torch.empty((0,1)).to(device)


dt = 0.01
t_end = 5
t_series = torch.arange(0, t_end, dt)

th = th_start
th_d = th_d_start
th_dd = th_dd_start

q_est = model.decoder(th_start, clockwise=is_clockwise_start)
q_d_est = (model.jacobian_dec(th_start, clockwise=is_clockwise_start) @ th_d_start.T).T
q_dd_est = torch.zeros((1,2))

q_real, q_d_real, q_dd_real = q_start, q_d_start, q_dd_start

is_clockwise = transforms.check_clockwise(q_start.squeeze(0))

for t in torch.arange(0, t_end, dt):
    t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
    print(t_string)

    q = q_real
    q_d = q_d_real

    th = model.encoder_vmap(q)
    th_d = (model.jacobian_enc(q) @ q_d.T).T

    q_hat = model.decoder_vmap(th, clockwise=model_cw)
    q_d_hat = (model.jacobian_dec(th) @ th_d.T).T
    
    """ Obtain Jacobian, dynamical matrices"""

    is_clockwise = transforms.check_clockwise(q_hat.squeeze(0))
    
    J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
    J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

    M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
    G_q_est = dynamics.add_spring_force_G_q(rp, q_hat, G_q_est, k_spring)
    A_q_est = dynamics.input_matrix(rp, q_hat.squeeze(0))


    """ Feed-forward simulation of the system, not on real dynamics """
    M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th, th_d, J_h_inv, J_h_inv_trans)
    M_th = torch.tensor([[rp["m2"], 0.], [0., rp["m2"]]]).to(device).requires_grad_(True)
    M_th = M_th * th/th
    C_th = torch.tensor([[0, 0.], [0., 0.]]).to(device).requires_grad_(True)
    A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans, device)

    Y = normal_form.calculate_Y(th, th_d, M_th, C_th, G_th, device)
    alpha, beta = normal_form.calculate_alpha_beta(th, th_d, M_th, C_th, G_th, A_th, Y)

    print("alpha:", alpha)
    print("beta:", beta)
    v = normal_form.calculate_v(Y, Y_des, K)
    u = normal_form.calculate_u(alpha, beta, v)
    print("Y:", Y)
    print("v:", v)
    print("u:", u)

    """ Recalculate "real system" based on modified matrices in theta-space. """

    



    """ Update the real system and apply latent control input. """

    M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q_real.squeeze(0), q_d_real.squeeze(0))
    G_q_real = dynamics.add_spring_force_G_q(rp, q_real, G_q_real, k_spring)
    A_q_real = dynamics.input_matrix(rp, q_real.squeeze(0))

    tau_q_real = A_q_real * u
    q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T)- G_q_real)).T  #  - C_damp @ ((q_d_real).T) 
    q_d_real = q_d_real + q_dd_real * dt
    q_real = q_real + q_d_real * dt
    q_real = transforms.wrap_to_pi(q_real)
    

    q_real_shifted = transforms.shift_q(q_real, clockwise=model_cw)
    th = model.encoder_vmap(q_real_shifted)
    q_est = model.decoder_vmap(th, clockwise=transforms.check_clockwise(q_real.squeeze(0)))
    q_est = transforms.wrap_to_pi(q_est)
    th_d = (model.jacobian_enc(q_real) @ q_d_real.T).T
    q_d_est = (model.jacobian_dec(th, clockwise=is_clockwise) @ th_d.T).T
    q_dd_est = torch.tensor([[0., 0.]]).requires_grad_().to(device) #TODO: Implement with Jacobian derivative


    """ Store data for plotting """

    th_series = torch.cat((th_series, th.detach()), dim=0)
    th_d_series = torch.cat((th_d_series, th_d.detach()), dim=0)
    th_dd_series = torch.cat((th_dd_series, th_dd.detach()), dim=0)
    
    q_real_series = torch.cat((q_real_series, q_real.detach()), dim=0)
    q_d_real_series = torch.cat((q_d_real_series, q_d_real.detach()), dim=0)
    q_dd_real_series = torch.cat((q_dd_real_series, q_dd_real.detach()), dim=0)

    q_est_series = torch.cat((q_est_series, q_est.detach()), dim=0)
    q_d_est_series = torch.cat((q_d_est_series, q_d_est.detach()), dim=0)
    q_dd_est_series = torch.cat((q_dd_est_series, q_dd_est.detach()), dim=0) 

    u_series = torch.cat((u_series, u.detach()), dim=0)
    Y_series = torch.cat((Y_series, Y.detach().T))
    print("")

    if torch.isnan(q[0,0]):
        break

    
    


Time: [0.0/5.0]
q: tensor([[-0.8498,  0.2159]], device='cuda:0', grad_fn=<UnsqueezeBackward0>)
q_d: tensor([[0., 0.]], device='cuda:0', grad_fn=<ToCopyBackward0>)
q_hat: tensor([[-0.8498,  0.2159]], device='cuda:0', grad_fn=<StackBackward0>)
q_d_hat: tensor([[0., 0.]], device='cuda:0', grad_fn=<PermuteBackward0>)
th: tensor([[7.1840, 1.8529]], device='cuda:0', grad_fn=<StackBackward0>)
th_d: tensor([[0., 0.]], device='cuda:0', grad_fn=<PermuteBackward0>)
M_th: tensor([[3., 0.],
        [0., 3.]], device='cuda:0', grad_fn=<DivBackward0>)
Hello (tensor([[0., 0.]], device='cuda:0', grad_fn=<AddBackward0>),)
C_th: tensor([[0., 0.],
        [0., 0.]], device='cuda:0', requires_grad=True)
dM0dthfull: tensor([[0., 0.]], device='cuda:0', grad_fn=<AddBackward0>)
dM1dthfull: tensor([[0., 0.]], device='cuda:0', grad_fn=<AddBackward0>)
dM0dth0: tensor(0., device='cuda:0', grad_fn=<SelectBackward0>)
dM1dth1: tensor(0., device='cuda:0', grad_fn=<SelectBackward0>)
ddM1ddth1: tensor(1.1921e-07, device

KeyboardInterrupt: 